In [24]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
pd.options.mode.chained_assignment = None  # default='warn'

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

df0 = pd.read_csv('monster-saves.csv')
df0 = df0.astype({'Book': 'category', 'Category': 'category', 'Legendary': 'category', 'Type': 'category', 'Size': 'category'})
df0['Str PB'] = df0['Str Prof']*df0['PB']
df0['Dex PB'] = df0['Dex Prof']*df0['PB']
df0['Con PB'] = df0['Con Prof']*df0['PB']
df0['Wis PB'] = df0['Wis Prof']*df0['PB']
df0['Int PB'] = df0['Int Prof']*df0['PB']
df0['Cha PB'] = df0['Cha Prof']*df0['PB']
print('Includes {} monsters in database'.format(len(df0.index.to_list())))

Includes 1781 monsters in database


In [25]:
# Setup template and html config for plotly figures.
import plotly.graph_objects as go

TFB_TEMPLATE = dict(
    layout=go.Layout(
        template='plotly_white',
        autosize=False,
        margin=dict(l=50, r=25, b=40, t=20, pad=4),
        font=dict(family='sans-serif', size=14),
        hovermode='closest',
        xaxis=dict(
            automargin=False,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            tickmode='linear',
            ticks='outside',
            minor=dict(
                tickmode='linear',
                ticks='outside',
            ),
            zeroline=False,
        ),
        yaxis=dict(
            automargin=False,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            ticks='outside',
            minor=dict(
                ticks='outside',
            ),
            zeroline=False,
        ),
        hoverlabel=dict(align='left'),
    )
)

TFB_CONFIG = {
    'responsive': True, 
    'showAxisDragHandles': False,
    'displaylogo': False,
    'displayModeBar': 'hover',
    'modeBarButtonsToRemove': [
        'select2d',
        'lasso2d',
        'zoom2d',
        'zoomIn2d',
        'zoomOut2d',
        'pan2d',
        'autoScale2d',
        'hoverClosestCartesian',
        'hoverCompareCartesian',
        'toggleSpikelines',
        'resetScale2d',
    ],
    'toImageButtonOptions': {
        'format': 'png', # one of png, svg, jpeg, webp
        'filename': 'tfb-plot',
        'height': 450,
        'width': 600,
        'scale': 2
    },
}

def plot_mean_fit(fig, df0, xCol, yCol, name, line_color):
    # plot mean
    dfM = df0[[xCol, yCol]].groupby(xCol).mean().reset_index()
    fig.add_trace(go.Scatter(
        x=dfM[xCol], y=dfM[yCol],
        mode='markers', 
        name=f'{name}',
        legendgroup=name, 
        line_color=line_color,
        hovertemplate='<b>' + name + '</b><br>' + 'CR %{x:,.0f}<br>' + 'Mean %{y:,.1f}<br>' + '<extra></extra>'
    ))
    coefs = np.polyfit(dfM[xCol], dfM[yCol], 1)
    poly = np.poly1d(coefs)
    fig.add_trace(go.Scatter(
        x=dfM[xCol], y=poly(dfM[xCol]),
        mode='lines', 
        name=f'{name}',
        legendgroup=name, 
        line_color=line_color,
        hoverinfo='skip',
        showlegend=False,
    ))
    
def save_fig_html(fig, format, name):
    file_name = f'./{name}.html'

    match format:
        case 'large':
            fig.update_traces(marker=dict(size=6))
            fig.update_layout(font=dict(size=14))
        case 'small':
            fig.update_traces(marker=dict(size=4))
            fig.update_layout(font=dict(size=10))

    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = f'plotly-div-{format}'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [ ]:
# Saving throw modifier
columns = ['Str Save','Dex Save','Con Save','Int Save','Wis Save','Cha Save']
colors = ['#d62728','#9467bd','#8c564b','#1f77b4','#2ca02c','#ff7f0e']

df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1, 30)]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='saving throw bonus',
            range=[-2,19],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

for column, color in zip(columns, colors):
    plot_mean_fit(fig, df1, 'CR', column, column, color)

# show figure
fig.show(config=TFB_CONFIG, width=600, height=450)
fig.update_layout(autosize=True)

# save figures
if SAVEFIGS:
    save_fig_html(fig, format='large', name='fig-saving-throw-modifier-vs-cr-large')
    save_fig_html(fig, format='small', name='fig-saving-throw-modifier-vs-cr-small')

In [26]:
# Ability score modifier
columns = ['Str Mod','Dex Mod','Con Mod','Int Mod','Wis Mod','Cha Mod']
colors = ['#d62728','#9467bd','#8c564b','#1f77b4','#2ca02c','#ff7f0e']

df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1, 30)]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='ability score modifier',
            range=[-2,11],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

for column, color in zip(columns, colors):
    plot_mean_fit(fig, df1, 'CR', column, column, color)

# show figure
fig.show(config=TFB_CONFIG, width=600, height=450)
fig.update_layout(autosize=True)

# save figures
if SAVEFIGS:
    save_fig_html(fig, format='large', name='fig-ability-score-modifier-vs-cr-large')
    save_fig_html(fig, format='small', name='fig-ability-score-modifier-vs-cr-small')

In [28]:
# Saving throw proficiency
columns = ['Str Prof','Dex Prof','Con Prof','Int Prof','Wis Prof','Cha Prof']
colors = ['#d62728','#9467bd','#8c564b','#1f77b4','#2ca02c','#ff7f0e']

df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1, 30)]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='saving throw proficiency probability',
            range=[-0.01,1.01],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

for column, color in zip(columns, colors):
    plot_mean_fit(fig, df1, 'CR', column, column, color)

# show figure
fig.show(config=TFB_CONFIG, width=600, height=450)
fig.update_layout(autosize=True)

# save figures
if SAVEFIGS:
    save_fig_html(fig, format='large', name='fig-saving-throw-proficiency-vs-cr-large')
    save_fig_html(fig, format='small', name='fig-saving-throw-proficiency-vs-cr-small')

In [29]:
# Saving throw proficiency bonus
columns = ['Str PB','Dex PB','Con PB','Int PB','Wis PB','Cha PB']
colors = ['#d62728','#9467bd','#8c564b','#1f77b4','#2ca02c','#ff7f0e']

df1 = df0[df0['Category'].isin(['generic','legendary','unique']) & df0['CR'].between(1, 30)]

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='saving throw bonus',
            range=[-2,11],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

for column, color in zip(columns, colors):
    plot_mean_fit(fig, df1, 'CR', column, column, color)

# show figure
fig.show(config=TFB_CONFIG, width=600, height=450)
fig.update_layout(autosize=True)

# save figures
if SAVEFIGS:
    save_fig_html(fig, format='large', name='fig-saving-throw-proficiency-bonus-vs-cr-large')
    save_fig_html(fig, format='small', name='fig-saving-throw-proficiency-bonus-vs-cr-small')